In [5]:
from statsmodels.tsa.stattools import grangercausalitytests
import csv
import requests
import pandas as pd
import numpy as np
import os

In [6]:
gtrends_data = pd.read_csv("gtrends_data/gtrends_weekly_01-01-12_10-29-18.csv", names=["ds", "y"])
gtrends_data['y'] = gtrends_data['y'].replace({"<1": "0"})

In [7]:
POLONIEX_OHLCV_BASEURL = 'https://poloniex.com/public?command=returnChartData&currencyPair='

"""
https://poloniex.com/public?command=returnChartData&currencyPair=BTC_POT&start=1435699200&end=9999999999&period=14400

"""

def get_ohlcv_poloniex(pair='USDT_ETH', start=1435699200, end=9999999999, period=900):
    """
    returns ohlcv data for poloniex as pandas dataframe
    convert to unix timestamp using https://coderstoolbox.net/unixtimestamp/
    :param pair: str pair on poloniex
    :param start: int unix timestamp of beginning time
    :param end: int unix timestamp of ending time
    :param period: int candle width in seconds
    :return: pandas df of ohlcv data from poloniex for specified pair, times, and period
    """
    query = POLONIEX_OHLCV_BASEURL + pair + '&start=' + str(start) + '&end=' + str(end) + '&period=' + str(period)
    resp = requests.get(query)

    if resp.status_code != 200:
        raise requests.ApiError('GET /tasks/ {}'.format(resp.status_code))

    return pd.DataFrame(resp.json())

In [8]:
df = get_ohlcv_poloniex(pair='USDT_BTC', start=0, end=9999999999, period=900)
df = df[['date','close','volume']]
df['ds'] = pd.to_datetime(df['date'], unit='s')
df['log'] = np.log(df['close'])
df['vol'] = df['log'].rolling(5).std()
df.dropna();

In [9]:
path = 'gtrends_data/'

In [11]:
for series in ['close']:
    for i in  [1,2,4,6,8]:
        for isAbs in [True, False]:
            data = df[series]
            if isAbs:
                data.apply(lambda x:abs(x))
            combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
            combined = combined.dropna(how='any')
            print (series)
            print (i)
            print (grangercausalitytests(combined, i))

close
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
parameter F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
{1: ({'ssr_ftest': (1.062981224047147, 0.3057225876793295, 78.0, 1), 'ssr_chi2test': (1.1038651172797296, 0.2934194346848047, 1), 'lrtest': (1.0964110462010694, 0.2950550809211057, 1), 'params_ftest': (1.0629812240470633, 0.3057225876793507, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb80e10>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb90080>, array([[0., 1., 0.]])])}
close
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
p

ssr based F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3
ssr based chi2 test:   chi2=9.1939  , p=0.0268  , df=3
likelihood ratio test: chi2=8.6971  , p=0.0336  , df=3
parameter F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.2254  , p=0.9233  , df_denom=69, df_num=4
ssr based chi2 test:   chi2=1.0192  , p=0.9069  , df=4
likelihood ratio test: chi2=1.0126  , p=0.9079  , df=4
parameter F test:         F=0.2254  , p=0.9233  , df_denom=69, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.2293  , p=0.9485  , df_denom=66, df_num=5
ssr based chi2 test:   chi2=1.3374  , p=0.9310  , df=5
likelihood ratio test: chi2=1.3259  , p=0.9322  , df=5
parameter F test:         F=0.2293  , p=0.9485  , df_denom=66, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.1248  , p=0.9929  , df_denom=63, df_num=6
ssr based chi2 test:   chi2=

close
8

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
parameter F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.2208  , p=0.3008  , df_denom=75, df_num=2
ssr based chi2 test:   chi2=2.6045  , p=0.2719  , df=2
likelihood ratio test: chi2=2.5630  , p=0.2776  , df=2
parameter F test:         F=1.2208  , p=0.3008  , df_denom=75, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3
ssr based chi2 test:   chi2=9.1939  , p=0.0268  , df=3
likelihood ratio test: chi2=8.6971  , p=0.0336  , df=3
parameter F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.2254  , p=0.9233

ssr based F test:         F=0.1406  , p=0.9946  , df_denom=60, df_num=7
ssr based chi2 test:   chi2=1.2302  , p=0.9902  , df=7
likelihood ratio test: chi2=1.2202  , p=0.9905  , df=7
parameter F test:         F=0.1406  , p=0.9946  , df_denom=60, df_num=7

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=0.1390  , p=0.9970  , df_denom=57, df_num=8
ssr based chi2 test:   chi2=1.4437  , p=0.9936  , df=8
likelihood ratio test: chi2=1.4298  , p=0.9938  , df=8
parameter F test:         F=0.1390  , p=0.9970  , df_denom=57, df_num=8
{1: ({'ssr_ftest': (1.062981224047147, 0.3057225876793295, 78.0, 1), 'ssr_chi2test': (1.1038651172797296, 0.2934194346848047, 1), 'lrtest': (1.0964110462010694, 0.2950550809211057, 1), 'params_ftest': (1.0629812240470633, 0.3057225876793507, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c173147f0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c17314828>, array([[0., 1., 

In [12]:
for series in ['volume']:
    for i in  [1,2,4,6,8]:
        for isAbs in [True, False]:
            data = df[series]
            if isAbs:
                data.apply(lambda x:abs(x))
            combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
            combined = combined.dropna(how='any')
            print (series)
            print (i)
            print (grangercausalitytests(combined, i))

volume
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8654  , p=0.3551  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.8987  , p=0.3431  , df=1
likelihood ratio test: chi2=0.8937  , p=0.3445  , df=1
parameter F test:         F=0.8654  , p=0.3551  , df_denom=78, df_num=1
{1: ({'ssr_ftest': (0.8654078107045098, 0.3551000272160141, 78.0, 1), 'ssr_chi2test': (0.8986927265008371, 0.3431324811972904, 1), 'lrtest': (0.8937438129383395, 0.34446478260020874, 1), 'params_ftest': (0.8654078107045222, 0.3551000272160091, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb80cc0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb85908>, array([[0., 1., 0.]])])}
volume
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8654  , p=0.3551  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.8987  , p=0.3431  , df=1
likelihood ratio test: chi2=0.8937  , p=0.3445  , df

ssr based F test:         F=0.3228  , p=0.8088  , df_denom=72, df_num=3
ssr based chi2 test:   chi2=1.0626  , p=0.7861  , df=3
likelihood ratio test: chi2=1.0556  , p=0.7878  , df=3
parameter F test:         F=0.3228  , p=0.8088  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.1850  , p=0.9454  , df_denom=69, df_num=4
ssr based chi2 test:   chi2=0.8367  , p=0.9335  , df=4
likelihood ratio test: chi2=0.8322  , p=0.9341  , df=4
parameter F test:         F=0.1850  , p=0.9454  , df_denom=69, df_num=4
{1: ({'ssr_ftest': (0.8654078107045098, 0.3551000272160141, 78.0, 1), 'ssr_chi2test': (0.8986927265008371, 0.3431324811972904, 1), 'lrtest': (0.8937438129383395, 0.34446478260020874, 1), 'params_ftest': (0.8654078107045222, 0.3551000272160091, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb80c88>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c17314a58>, array([[0., 1.

likelihood ratio test: chi2=0.9405  , p=0.9672  , df=5
parameter F test:         F=0.1622  , p=0.9755  , df_denom=66, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.1607  , p=0.9861  , df_denom=63, df_num=6
ssr based chi2 test:   chi2=1.1633  , p=0.9787  , df=6
likelihood ratio test: chi2=1.1545  , p=0.9791  , df=6
parameter F test:         F=0.1607  , p=0.9861  , df_denom=63, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.0767  , p=0.9992  , df_denom=60, df_num=7
ssr based chi2 test:   chi2=0.6707  , p=0.9986  , df=7
likelihood ratio test: chi2=0.6677  , p=0.9986  , df=7
parameter F test:         F=0.0767  , p=0.9992  , df_denom=60, df_num=7

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=0.0742  , p=0.9997  , df_denom=57, df_num=8
ssr based chi2 test:   chi2=0.7711  , p=0.9993  , df=8
likelihood ratio test: chi2=0.7671  , p=0.9993  , df=8
parameter F test:         F=0.0742  , p=0.999

likelihood ratio test: chi2=0.6677  , p=0.9986  , df=7
parameter F test:         F=0.0767  , p=0.9992  , df_denom=60, df_num=7

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=0.0742  , p=0.9997  , df_denom=57, df_num=8
ssr based chi2 test:   chi2=0.7711  , p=0.9993  , df=8
likelihood ratio test: chi2=0.7671  , p=0.9993  , df=8
parameter F test:         F=0.0742  , p=0.9997  , df_denom=57, df_num=8
{1: ({'ssr_ftest': (0.8654078107045098, 0.3551000272160141, 78.0, 1), 'ssr_chi2test': (0.8986927265008371, 0.3431324811972904, 1), 'lrtest': (0.8937438129383395, 0.34446478260020874, 1), 'params_ftest': (0.8654078107045222, 0.3551000272160091, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb80eb8>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c173159b0>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (0.404608654308501, 0.6686854868875393, 75.0, 2), 'ssr_chi2test': (0.8631651291914688, 0.649480436

In [13]:
for series in ['vol']:
    for i in  [1,2,4,6,8]:
        for isAbs in [True, False]:
            data = df[series]
            if isAbs:
                data.apply(lambda x:abs(x))
            combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
            combined = combined.dropna(how='any')
            print (series)
            print (i)
            print (grangercausalitytests(combined, i))

vol
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.3150  , p=0.5746  , df=1
likelihood ratio test: chi2=0.3144  , p=0.5750  , df=1
parameter F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1
{1: ({'ssr_ftest': (0.3030235820043891, 0.5836075151727691, 76.0, 1), 'ssr_chi2test': (0.3149850391887729, 0.5746371553887759, 1), 'lrtest': (0.3143587566826227, 0.5750177137771262, 1), 'params_ftest': (0.30302358200433493, 0.583607515172806, 76.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c17310198>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c17310d68>, array([[0., 1., 0.]])])}
vol
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.3150  , p=0.5746  , df=1
likelihood ratio test: chi2=0.3144  , p=0.5750  , df=1
para

ssr based F test:         F=0.1527  , p=0.9878  , df_denom=61, df_num=6
ssr based chi2 test:   chi2=1.1118  , p=0.9810  , df=6
likelihood ratio test: chi2=1.1035  , p=0.9814  , df=6
parameter F test:         F=0.1527  , p=0.9878  , df_denom=61, df_num=6
{1: ({'ssr_ftest': (0.3030235820043891, 0.5836075151727691, 76.0, 1), 'ssr_chi2test': (0.3149850391887729, 0.5746371553887759, 1), 'lrtest': (0.3143587566826227, 0.5750177137771262, 1), 'params_ftest': (0.30302358200433493, 0.583607515172806, 76.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c0fb80278>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c173150f0>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (0.15164511843015696, 0.8595631608954024, 73.0, 2), 'ssr_chi2test': (0.32406354075485594, 0.8504141861313792, 2), 'lrtest': (0.3233922124984474, 0.8506996875813966, 2), 'params_ftest': (0.15164511842940315, 0.859563160896053, 73.0, 2.0)}, [<statsmodels.regression.linear_m

vol
8

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.3150  , p=0.5746  , df=1
likelihood ratio test: chi2=0.3144  , p=0.5750  , df=1
parameter F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1516  , p=0.8596  , df_denom=73, df_num=2
ssr based chi2 test:   chi2=0.3241  , p=0.8504  , df=2
likelihood ratio test: chi2=0.3234  , p=0.8507  , df=2
parameter F test:         F=0.1516  , p=0.8596  , df_denom=73, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.2846  , p=0.8363  , df_denom=70, df_num=3
ssr based chi2 test:   chi2=0.9393  , p=0.8159  , df=3
likelihood ratio test: chi2=0.9336  , p=0.8173  , df=3
parameter F test:         F=0.2846  , p=0.8363  , df_denom=70, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.3058  , p=0.8731  